## Full Test

In [2]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import datetime
from columns import *
from raidar_credentials import *

In [147]:
date_from = input("Export encounters since (mm/dd/yyyy): ")
date_from = "&since=" + str(time.mktime(datetime.datetime.strptime(date_from, "%m/%d/%Y").timetuple())).split('.')[0]
    # the ecounter list consists of several Json files. This initiates the url for the first one.
url = "https://www.gw2raidar.com/api/v2/encounters?limit=1000000" + date_from
    
    # initiate an empty list
url_ids = []
    
    # issue request
r = requests.get(url, headers = headers)

    # extract the url ID from the json and append to list
for encounter in r.json()['results']:
    url_ids.append(encounter['url_id'])  

# append full url to the IDs
url_ids = ['https://gw2raidar.com/encounter/' + id + '.json' for id in url_ids]


Export encounters since (mm/dd/yyyy): 01/01/2000


Get the encounter list

In [3]:
login = {
    'username': (None, username),
    'password': (None, password),
}

token = requests.post('https://www.gw2raidar.com/api/v2/token', files=login)

authentication = 'token ' + token.json()['token']
headers = {'Authorization': authentication}

# same as above function, but prompts keyboard input from the user
date_from = input("Export encounters since (mm/dd/yyyy): ")
date_from = "&since=" + str(time.mktime(datetime.datetime.strptime(date_from, "%m/%d/%Y").timetuple())).split('.')[0]
    # the ecounter list consists of several Json files. This initiates the url for the first one.
url = "https://www.gw2raidar.com/api/v2/encounters?limit=1000000" + date_from
    
    # initiate an empty list
url_ids = []
    
    # issue request
r = requests.get(url, headers = headers)

    # extract the url ID from the json and append to list
for encounter in r.json()['results']:
    url_ids.append(encounter['url_id'])  

# append full url to the IDs
# url_ids = ['https://gw2raidar.com/encounter/' + id + '.json' for id in url_ids]

Export encounters since (mm/dd/yyyy): 01/01/2000


Define the encounter class

In [7]:
class Encounter:
    def __init__(self, url_id):
        self.url = 'https://gw2raidar.com/encounter/' + url_id +'.json'
        self.url_id = url_id
    
    def get_json(self, url):
        encounter_response = requests.get(url, headers = headers)
        return encounter_response
    
    def get_df(self, encounter_response):
        encounter_df = pd.DataFrame()
        
        try:
            for subsquad in encounter_response.json()['encounter']['parties']:
                subsquad_df = json_normalize(encounter_response.json()['encounter']['parties'][subsquad]['members'])
                
                subsquad_df['subsquad'] = subsquad
                subsquad_df['Boss'] = encounter_response.json()['encounter']['name']
                subsquad_df['id'] = encounter_response.json()['encounter']['id']
                subsquad_df['Duration'] = encounter_response.json()['encounter']['duration']
                subsquad_df['success'] = encounter_response.json()['encounter']['success']
                subsquad_df['uploaded_at'] = encounter_response.json()['encounter']['uploaded_at']
                subsquad_df['URL ID'] = self.url_id
                encounter_df = encounter_df.append(subsquad_df, ignore_index = True)
        
        except JSONDecodeError:
            print('JSON error at ' + url_id)
        except Exception as e:
            print(e)
        
        return encounter_df                       

Progress bar on loop

In [11]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

Data Scrape and compile

In [13]:
%%time
# current run time is about 2.6 seconds per encounter. Implement parallel
master_df = pd.DataFrame()
for pull in log_progress(url_ids[:1389], every = 1):
        pull = Encounter(pull)
        pull_json = pull.get_json(pull.url)
        pull_df = pull.get_df(pull_json)
        master_df = master_df.append(pull_df, ignore_index = True)

VBox(children=(HTML(value=''), IntProgress(value=0, max=1389)))

ConnectionError: HTTPSConnectionPool(host='gw2raidar.com', port=443): Max retries exceeded with url: /encounter/ConstitutionsTrendQualityGrabbingMystery.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001EBFE963630>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [6]:
master_df.to_csv('test_data.csv', index = False)

### Navigation & Notes

In [ ]:
## Subsquads
# phase level stats
test.json()['encounter']['parties']['1']['phases']

# subsquad cleave dps for all phases
test.json()['encounter']['parties']['1']['phases']['All']['actual']['dps']

# subsquad boss dps for all phases
test.json()['encounter']['parties']['1']['phases']['All']['actual_boss']['dps']

# scholar uptime
test.json()['encounter']['parties']['1']['phases']['All']['actual_boss']['scholar']

# crit percent
test.json()['encounter']['parties']['1']['phases']['All']['actual_boss']['crit']

# flank percent
test.json()['encounter']['parties']['1']['phases']['All']['actual_boss']['flanking']

# number of subsquad members
len(test.json()['encounter']['parties']['1']['members'])

In [45]:
## Players
# character name
test.json()['encounter']['parties']['1']['members'][0]['name']

# account name
test.json()['encounter']['parties']['1']['members'][0]['account']

# profession
test.json()['encounter']['parties']['1']['members'][0]['profession']

# elite spec
test.json()['encounter']['parties']['1']['members'][0]['elite']

# archetype
test.json()['encounter']['parties']['1']['members'][0]['archetype']

# death and downs dict
test.json()['encounter']['parties']['1']['members'][0]['phases']['All']['events']

# buffs dict
test.json()['encounter']['parties']['1']['members'][0]['phases']['All']['buffs']

# mechanics dict
test.json()['encounter']['parties']['1']['members'][0]['phases']['All']['mechanics']

# boss dps dict
test.json()['encounter']['parties']['1']['members'][0]['phases']['All']['actual_boss']

# cleave dps dict
test.json()['encounter']['parties']['1']['members'][0]['phases']['All']['actual']


4

### Define Class to retrieve basic encounter data

## testing/scrap code

In [14]:
# This processes the data in the class and then appends it to the data frame. It runs faster than the other version but
# encounters a key error on older logs
class Encounter:
    def __init__(self, url):
        self.url = url
    
    def get_json(self, url):
        encounter_response = requests.get(url, headers = headers)
        return encounter_response
    
    def get_df(self, encounter_response):
        encounter_df = pd.DataFrame()
        for subsquad in encounter_response.json()['encounter']['parties']:
            subsquad_df = json_normalize(encounter_response.json()['encounter']['parties'][subsquad]['members'])
            
            if encounter_response.json()['encounter']['name'] == 'Vale Guardian':
                subsquad_df = subsquad_df[columns + vg_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Gorseval':
                subsquad_df = subsquad_df[columns + gors_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Sabetha':
                subsquad_df = subsquad_df[columns]
            elif encounter_response.json()['encounter']['name'] == 'Slothasor':
                subsquad_df = subsquad_df[columns + sloth_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Matthias':
                subsquad_df = subsquad_df[columns + matt_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Keep Construct':
                subsquad_df = subsquad_df[columns + kc_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Xera':
                subsquad_df = subsquad_df[columns + xera_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Cairn':
                subsquad_df = subsquad_df[columns + cairn_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Mursaat Overseer':
                subsquad_df = subsquad_df[columns + mo_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Samarog':
                subsquad_df = subsquad_df[columns + sam_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Deimos':
                subsquad_df = subsquad_df[columns + deimos_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Soulless Horror':
                subsquad_df = subsquad_df[columns + sh_mechanics]
            elif encounter_response.json()['encounter']['name'] == 'Dhuum':
                subsquad_df = subsquad_df[columns + dhuum_mechanics]
            else:
                subsquad_df = subsquad_df[columns]
            
            subsquad_df['subsquad'] = subsquad
            subsquad_df['Boss'] = encounter_response.json()['encounter']['name']
            subsquad_df['id'] = encounter_response.json()['encounter']['id']
            subsquad_df['Duration'] = encounter_response.json()['encounter']['duration']
            subsquad_df['success'] = encounter_response.json()['encounter']['success']
            subsquad_df['uploaded_at'] = encounter_response.json()['encounter']['uploaded_at']
            encounter_df = encounter_df.append(subsquad_df, ignore_index = True)
        return encounter_df                       